# Text Representation

In [2]:
import pandas as pd

In [3]:
# Fetch preprocessed data
model_data = pd.read_csv("data/keywords.csv")
model_data.head()

,Id,Name,keywords
0,1030195,morning star,morning_star h._rider_haggard 1st_world_libra...
1,1090099,kitty's class day and other stories,kitty's_class_day_and_other_stories louisa_ma...
2,1013522,the art of french kissing,the_art_of_french_kissing kristin_harmel 5_sp...
3,1003725,"comet in moominland (the moomins, #2)","comet_in_moominland the_moomins,_#2 tove_janss..."
4,1099901,the boy who cried horse,the_boy_who_cried_horse terry_deary a_&_c_bla...


---
### Vectorize the keywords summary using TF-IDF

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_encoding = tfidf.fit_transform(model_data["keywords"])

In [9]:
tfidf.get_feature_names_out()

array(['000', '000_cooking_methods_and_recipes', '000_greatest_works',
       ..., 'زكريا_تامر', 'مقالات', '鬼の子chapter'], dtype=object)